# Hente hemmeligheter fra secret manager

In [ ]:
from google.cloud import secretmanager
from google.oauth2 import service_account

In [ ]:
credentials = service_account.Credentials.from_service_account_file('/home/jupyter/sa_key.json')

In [ ]:
secrets = secretmanager.SecretManagerServiceClient(credentials=credentials)

In [ ]:
project_name = "knada-gcp"
secret_name = "vebjorn-rekkebo-bac1"
version = "latest"

resource_name = f"projects/{project_name}/secrets/{secret_name}/versions/{version}"
secret = secrets.access_secret_version(name=resource_name)

In [ ]:
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

In [ ]:
import psycopg2

import pandas as pd
import datetime

from sqlalchemy import create_engine

In [ ]:
method = "postgresql+psycopg2"
host = "A01DBVL028.adeo.no:5432"
#host = "10.53.15.222:5432"
db_name = "pensjon-psak"

alchemyEngine = create_engine(f'{method}://{secrets["PSAK_POSTGRES_USER"]}:{secrets["PSAK_POSTGRES_PASSWORD"]}@{host}/{db_name}')
dbConnection = alchemyEngine.connect()

In [ ]:
df_pref = pd.read_sql(
    """
    select bruk_nytt_design
    from t_saksbeh_preferanser;
    """,
    dbConnection)

In [ ]:
df_pref.head()

In [ ]:
df = (
    pd.DataFrame([
        df_pref[df_pref.bruk_nytt_design == True].count(),
        df_pref[df_pref.bruk_nytt_design == False].count(),
        df_pref[pd.isna(df_pref.bruk_nytt_design)].count()
    ])
    .T
    .rename(columns={0: "Nytt", 1: "Gammelt", 2: "Uspesifisert"})
)

df.head()

In [ ]:
df["Andel_nytt"] = (df.Nytt / df.sum(axis=1)).apply(lambda x: round(x, 3))
df["Uttrekk_tidspunkt"] = datetime.datetime.utcnow()

In [ ]:
df.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig, SchemaField, enums

table_id = f'pensjon-saksbehandli-prod-1f83.saksbehandling_psak.nytt_gammelt_design'
job_config = LoadJobConfig(
    schema = [
        SchemaField("Uspesifisert", enums.SqlTypeNames.INTEGER),
        SchemaField("Gammelt", enums.SqlTypeNames.INTEGER),
        SchemaField("Nytt", enums.SqlTypeNames.INTEGER),
        SchemaField("Uttrekk_tidspunkt", enums.SqlTypeNames.TIMESTAMP),
        SchemaField("Andel_nytt", enums.SqlTypeNames.FLOAT),
    ],
    write_disposition="WRITE_APPEND",
)

client = Client(project="pensjon-saksbehandli-prod-1f83", credentials=credentials)

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()